# Finetuning supervisé avec SFTTrainer

Ce notebook montre comment finetuner le modèle `HuggingFaceTB/SmolLM2-135M` en utilisant le `SFTTrainer` de la bibliothèque `trl`. Les cellules du notebook s'exécutent et vont finetuner le modèle. Vous pouvez choisir votre difficulté en essayant différents jeux de données.
<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercice : Finetuning de SmolLM2 avec SFTTrainer</h2>
    <p>Prenez un jeu de données provenant du Hub d'Hugging Face et finetuné un modèle sur dessus. </p> 
    <p><b>Niveaux de difficulté</b></p>
    <p>🐢 Utilisez le jeu de données `HuggingFaceTB/smoltalk`</p>
    <p>🐕 Essayez le jeu de données `bigcode/the-stack-smol` et finetunez un modèle de génération de code sur un sous-ensemble spécifique `data/python`</p>
    <p>🦁 Sélectionnez un jeu de données en rapport avec un cas d'utilisation réel qui vous intéresse</p>
</div>

In [ ]:
# Installer les prérequis dans Google Colab
# !pip install transformers datasets trl huggingface_hub

# S'authentifier sur Hugging Face
from huggingface_hub import login

login()

# Pour plus de facilité, vous pouvez créer une variable d'environnement contenant votre jeton de hub sous la forme HF_TOKEN

In [ ]:
# Importer les bibliothèques nécessaires
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Charger le modèle et le tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Définir le format de chat
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Définir le nom du finetuning à sauvegarder et/ou à télécharger
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# Générer avec le modèle de base

Ici, nous allons essayer le modèle de base qui n'a pas de gabarit de chat. 

In [ ]:
# Testons le modèle de base avant l'entraînement
prompt = "Write a haiku about programming"

# Format avec gabarit
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Générer une réponse
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## Préparation du jeu de données

Nous allons charger un échantillon du jeu de données et le formater pour l'entraînement. Le jeu de données doit être structuré avec des paires entrée-sortie, où chaque entrée est une instruction et la sortie est la réponse attendue du modèle.

**TRL va formater les messages d'entrée en se basant sur les gabarits de chat du modèle.** Ils doivent être représentés sous la forme d'une liste de dictionnaires avec les clés : `role` et `content`.

In [ ]:
# Chargement d'un échantillon du jeu de données
from datasets import load_dataset

# TODO : définir votre jeu de données et votre configuration en utilisant les paramètres path et name
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [ ]:
# TODO : 🦁 Si votre jeu de données n'est pas dans un format que TRL peut convertir en gabarit de chat, vous devrez le traiter. Reportez-vous au [module](../chat_templates.md)

## Configurer le SFTTrainer

Le `SFTTrainer` est configuré avec différents paramètres qui contrôlent le processus d'apprentissage. Ceux-ci incluent le nombre d'étapes d'entraînement, la taille de batch, le taux d'apprentissage et la stratégie d'évaluation. Ajustez ces paramètres en fonction de vos besoins spécifiques et de vos ressources de calcul.

In [ ]:
# Configurer le SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Ajuster en fonction de la taille du jeu de données et de la durée d'entraînement souhaitée
    per_device_train_batch_size=4,  # Régler en fonction de la capacité de mémoire de votre GPU
    learning_rate=5e-5,  # Point de départ commun pour le finetuning
    logging_steps=10,  # Fréquence d'enregistrement des métriques d'entraînement
    save_steps=100,  # Fréquence de sauvegarde des checkpoints du modèle
    evaluation_strategy="steps",  # Évaluer le modèle à intervalles réguliers
    eval_steps=50,  # Fréquence de l'évaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Utiliser MPS pour un entraînement à précision mixte
    hub_model_id=finetune_name,  # Définissez un nom unique pour votre modèle
)

# Initialiser le SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO : 🦁 🐕 aligner les paramètres de SFTTrainer avec le jeu de données que vous avez choisi. 
# Par exemple, si vous utilisez le jeu de données `bigcode/the-stack-smol`, vous devrez choisir la colonne `content`

## Entraînement du modèle

Une fois le Trainer configuré, nous pouvons maintenant procéder à l'entraînement du modèle. Le processus d'entraînement consiste à itérer sur le jeu de données, à calculer la perte et à mettre à jour les paramètres du modèle afin de minimiser cette perte.

In [ ]:
# Entraîner le modèle
trainer.train()

# Sauvegarder le modèle
trainer.save_model(f"./{finetune_name}")

In [ ]:
trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercice bonus : Générer avec un modèle finetuné</h2>
    <p>🐕 Utiliser le modèle finetuné pour générer une réponse, comme dans l'exemple de base</p>
</div>

In [ ]:
# Tester le modèle finetuné sur la même instruction

# Testons le modèle de base avant l'entraînement
prompt = "Write a haiku about programming"

# Format avec gabarit
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Générer une réponse
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO : utiliser le modèle finetuné pour générer une réponse, comme dans l'exemple de base.

## 💐 Vous avez terminé !

Ce *notebook* fournit un guide étape par étape pour finetuner le modèle `HuggingFaceTB/SmolLM2-135M` en utilisant le `SFTTrainer`. En suivant ces étapes, vous pouvez adapter le modèle pour effectuer des tâches spécifiques plus efficacement. Si vous voulez continuer à travailler sur ce cours, voici quelques étapes que vous pouvez essayer :

- Essayez ce *notebook*  avec un niveau de difficulté plus élevé
- Examiner la PR d'un collègue
- Améliorez le matériel de cours par le biais d'une *issue* ou d'une PR.